QR Code Based Smart Attendance System 

In [ ]:
import cv2 
import numpy  as np 
from pyzbar.pyzbar import decode 
from cvzone import cornerRect, putTextRect
import pandas as pd  
import re 
import time
import os 
from datetime import datetime 
from cvzone.SerialModule import SerialObject
from cvzone.FPS import FPS 


fpsReader = FPS(avgCount=30)

cap = cv2.VideoCapture(0)

flag = True
textFlag = True 
counter = 0


# Read the root database containing student information
root_database_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\QR Attendance Database\rootDatabase.xlsx'
root_db = pd.read_excel(root_database_path)
# Convert the 'ID' column to a list
id_list = root_db['ID'].tolist()

individual_records_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\QR Attendance Database\Individual_Records'
daily_records_path = r'C:\Users\hoque\Desktop\OpenCV\VSCode\QR Attendance Database\Daily_Records'

last_scan_time = {}

def create_daily_file(date):
    # Create a new Excel file for the current date if it doesn't exist
    daily_file_name = os.path.join(daily_records_path, f'{date}.xlsx')
    try:
        daily_df = pd.read_excel(daily_file_name)
    except FileNotFoundError:
        # Create a new dataframe with the required columns
        daily_df = root_db[['No.', 'ID', 'Name', 'DoB']]
        daily_df['Entry'] = 'Nill'
        daily_df['Exit'] = 'Nill'
        daily_df.to_excel(daily_file_name, index=False)
    return daily_file_name

def log_entry_exit(id, timestamp):

    # Check if the ID has been scanned within the last 60 seconds
    if id in last_scan_time and (timestamp - last_scan_time[id]).total_seconds() < 10:
        print(f"ID {id}  Try again in 10 seconds.")
        return False 
    
    student = root_db[root_db['ID'] == id].iloc[0]
    daily_file_name = create_daily_file(timestamp.strftime('%d-%m-%Y'))
    
    # Update the daily attendance
    daily_df = pd.read_excel(daily_file_name)

    # Ensure the 'Entry' and 'Exit' columns are of type str
    daily_df['Entry'] = daily_df['Entry'].astype(str)
    daily_df['Exit'] = daily_df['Exit'].astype(str)
    
    #Check if the newly added id is present

    id_current = daily_df['ID'].tolist()
    
    if id not in id_current: 
        count = len(daily_df) + 1
        name  = root_db.loc[root_db['ID'] == id, 'Name'].values[0]
        dob   = root_db.loc[root_db['ID'] == id, 'DoB'].values[0]

        new_entry = pd.DataFrame({'No.': [count], 'ID': [id], 'Name': [name], 'DoB': [dob], 'Entry': ['Nill'], 'Exit': ['Nill']})
        daily_df = pd.concat([daily_df, new_entry], ignore_index=True)
        daily_df.to_excel(daily_file_name, index=False)


    row_index = daily_df[daily_df['ID'] == id].index[0]
    
    if daily_df.at[row_index, 'Entry'] == 'Nill':
        daily_df.at[row_index, 'Entry'] = timestamp.strftime('%H:%M:%S')
    else:
        daily_df.at[row_index, 'Exit'] = timestamp.strftime('%H:%M:%S')
    daily_df.to_excel(daily_file_name, index=False)

    # Update the individual attendance
    individual_file_name = os.path.join(individual_records_path, f'{id}.xlsx')
    try:
        individual_df = pd.read_excel(individual_file_name)
    except FileNotFoundError:
        individual_df = pd.DataFrame(columns=['Date', 'Entry', 'Exit'])

    # Ensure the 'Entry' and 'Exit' columns are of type str
    individual_df['Entry'] = individual_df['Entry'].astype(str)
    individual_df['Exit'] = individual_df['Exit'].astype(str)
    
    date_str = timestamp.strftime('%d-%m-%Y')
    if date_str in individual_df['Date'].values:
        row_index = individual_df[individual_df['Date'] == date_str].index[0]
        individual_df.at[row_index, 'Exit'] = timestamp.strftime('%H:%M:%S')
    else:
        new_entry = pd.DataFrame({'Date': [date_str], 'Entry': [timestamp.strftime('%H:%M:%S')], 'Exit': [None]})
        individual_df = pd.concat([individual_df, new_entry], ignore_index=True)
    individual_df.to_excel(individual_file_name, index=False)

    # Update the last scan time for the ID
    last_scan_time[id] = timestamp
    
    return True


#print(id_list)

#Configure the background image 
imgBackground = cv2.imread(r"C:\Users\hoque\Desktop\OpenCV\VSCode\Files\background2.jpg")
imgBackground = cv2.resize(imgBackground, (1000,500))


while True: 
    
    imgBackground = cv2.imread(r"C:\Users\hoque\Desktop\OpenCV\VSCode\Files\background2.jpg")
    imgBackground = cv2.resize(imgBackground, (1000,500))

    success, img = cap.read()
    H,W,_ = img.shape
    roiy1, roiy2, roix1, roix2 = int(H*0.3), int(H-0.3*H) ,int(W*0.3), int(W-0.3*W) 
    x, y, w, h = roix1, roiy1, roix2-roix1, roiy2-roiy1 
    #cornerRect(img, [x,y,w,h])
    roi = img[roiy1:roiy2, roix1:roix2]

    #Graphical Addition 
    imgroi= cv2.resize(roi, (832-570,393-116))
    imgBackground[116:393, 570:832] = imgroi 

    results = decode(roi)

    if results: 
            for barcode in results: 

                encodedData = barcode.data.decode('utf-8')
                code = re.search(r'\d+', encodedData).group()
                code = int(code)
                if flag and counter==0: 
                    counter = 1
                    print(encodedData)
                    flag = False
                    if code in id_list:
                         
                         name  = root_db.loc[root_db['ID'] == code, 'Name'].values[0]
                         timestamp = datetime.now()
                         textFlag = log_entry_exit(code, timestamp)
                         #color = (0,210,0)

                         if not textFlag: 
                            textr =  "WAIT 10 SEC"
                            textFlag =True
                            color = (0,0,210)
                            #flag=True   ##For auto authorization after 10sec
                            #counter = 0  ##For auto authorization after 10sec
                            #print("CheckPoint")
                         else: 
                            textr = "AUTHORIZED"
                            color = (0,220,0)
                         
                         
                         #print("Success")
                    else: 
                         color = (0,0,255)
                         textr =  "UNAUTHORIZED"
                         #print("Failure")

                if counter!=0:
                    counter+=1
                    if counter>12:
                            counter=0

                pts = np.array([barcode.polygon], np.int32)
                pts = pts.reshape((-1,1,2))

                pts[:, 0, 0] += roix1  # Adjust x coordinates
                pts[:, 0, 1] += roiy1  # Adjust y coordinates

                # Calculate scale factors
                scale_x = (832 - 570) / (roix2 - roix1)
                scale_y = (393 - 116) / (roiy2 - roiy1)

                # Adjust the points
                pts2 = np.zeros_like(pts)
                pts2[:, 0, 0] = (pts[:, 0, 0] - roix1) * scale_x + 570
                pts2[:, 0, 1] = (pts[:, 0, 1] - roiy1) * scale_y + 116

               
                # putTextRect(img, text=textr, pos=(int(W*0.7), 30), scale=1.5, thickness=2, colorR=color)
                # putTextRect(imgBackground, text=textr, pos=(int(W*0.7), 30), scale=1.5, thickness=2, colorR=color)

                putTextRect(img, text=textr, pos=(int(roix1+55), roiy1-15), scale=1.5, thickness=2, colorR=color, colorB=(0,0,0), border=2)
                putTextRect(imgBackground, text=textr, pos=(int(570+60), 116-20), scale=1.5, thickness=2, colorR=color, colorB=(0,0,0), border=2)

                if textr != "UNAUTHORIZED":
                    putTextRect(img, text=name, pos=(int(roix1+55), roiy1-55), scale=1.5, thickness=2, colorR=color, colorB=(0,0,0), border=2)
                    putTextRect(imgBackground, text=name, pos=(int(570+60), 116-60), scale=1.5, thickness=2, colorR=color, colorB=(0,0,0), border=2)

                cv2.polylines(img, [pts], True, color=color, thickness=3)
                cv2.polylines(imgBackground, [pts2], True, color=color, thickness=3)
                #flag = False
                
    else: 
        flag = True


    
    #fpsReader.update returns the current FPS and the updated image
    fps, img = fpsReader.update(img, pos=(20, 50),bgColor=(0, 0, 255), textColor=(255, 255, 255), scale=1.3, thickness=2)

    
    putTextRect(imgBackground, text="SCAN QR", pos=(int(570+75), 393+30), scale=1.5, thickness=2, colorR=(255,0,0))
    cornerRect(imgBackground, [570-5,116-5,262+5,277+5], rt=6, t=10,colorR=(0,0,0),colorC=(255,0,0),l=35)
    imgBackground= cv2.resize(imgBackground, (800,480))
    cv2.imshow("Grphics Added", imgBackground )
    
    cornerRect(img, [x,y,w,h], rt=2)
    putTextRect(img, text="SCAN QR", pos=(int(roix1+72), roiy2+30), scale=1.5, thickness=2, colorR=(255,0,0), colorB=(0,0,0), border=3)
    cv2.imshow("QR Scanner", img)
   

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

    
